In [1]:
import pandas as pd
import numpy as np
import glob
import os

First we filter the original csv to keep rows that are relevant

In [2]:
# Define the input directory and output file
file_path = 'data/saccade/SaccadeTable_easy1_second.csv'  # Update with your directory path
output_file = 'debugging.csv'

In [11]:
# Initialize an empty list to hold all rows for the combined DataFrame
combined_rows = []

# Iterate over all CSV files in the specified directory

# Load each CSV file into a DataFrame, skipping the first 6 rows
df = pd.read_csv(file_path, skiprows=6, keep_default_na=False)

# Drop specific columns if they exist in the DataFrame
columns_to_drop = ["Respondent Gender", "Respondent Age", "Respondent Group", "Stimulus Label"]
df = df.drop([col for col in columns_to_drop if col in df.columns], axis='columns')

#Drop participants where ET is lower than 85
df.drop(df[(df['Study Name'].str.contains("easy 1", na=False)) & (df["Respondent Name"]==20020)].index, inplace=True)

# Drop rows based on conditions
df = df[df["Gaze Calibration"] != "Poor"]
df = df[df["AOI Label"] != "NA"]
# Check again if "Respondent Name" still contains 20022
if (df["Respondent Name"] == 20022).any():
    print("Respondent 20022 still present after filtering AOI Label at", file_path)
else:
    print("Respondent 20022 not present after filtering AOI Label.")

    
# Initialize a dictionary to collect participant-specific data for calculating statistics
participants_collection = {}

# Collecting data for each participant
for index, row in df.iterrows():
    study = row["Study Name"]
    participant = row["Respondent Name"]
    saccade_amp = row["Saccade Amplitude"]
    saccade_peak_vel = row["Saccade Peak Velocity"]
    
    #Fix naming convention
    df.loc[index, "Study Name"] = "Bridge 1"

    if participant not in participants_collection:
        participants_collection[participant] = {
            'Saccade Amplitude': [],
            'Saccade Peak Velocity': []
        }
    
    participants_collection[participant]["Saccade Amplitude"].append(saccade_amp)
    participants_collection[participant]["Saccade Peak Velocity"].append(abs(saccade_peak_vel))

# Calculate mean and std for each participant
participants_stats = {}

def get_stats(data):
    cleaned_data = [float(x) for x in data if x != "NA"]
    mean_value = np.mean(cleaned_data)
    std_dev = np.std(cleaned_data)
    return [mean_value, std_dev]

for participant, data in participants_collection.items():
    participants_stats[participant] = {
        "Saccade Amplitude": get_stats(data["Saccade Amplitude"]),
        "Saccade Peak Velocity": get_stats(data["Saccade Peak Velocity"])
    }
print(participants_collection[20022])
print(participants_stats[20022])
    
    #Now, filter outliers.
def check_range(stats, data):
    if data == "NA":
        return False
    mean, std = stats
    
    # Define the range for 3.5 standard deviations
    lower_bound = mean - 3.5 * std
    upper_bound = mean + 3.5 * std
    
    return lower_bound <=abs(float(data)) <= upper_bound

    
filtered_row ={}
filtered_row["Saccade Amplitude"] = []
filtered_row["Saccade Peak Acceleration"] =[]
filtered_row["Saccade Peak Deceleration"] =[]
filtered_row["Saccade Peak Velocity"] =[]
sacacade_amplitude_count = 0
saccade_peak_velocity_count = 0

for index, row in df.iterrows():
    participant= row["Respondent Name"]
    saccade_amp = row["Saccade Amplitude"]
    saccade_peak_vel = row["Saccade Peak Velocity"]
    # saccade_peak_ac = row["Saccade Peak Acceleration"]
    # sccade_peak_dec = row["Saccade Peak Deceleration"]
    
    participant_stat_amp = participants_stats[participant]["Saccade Amplitude"]
    participant_stat_vel = participants_stats[participant]["Saccade Peak Velocity"]
    # participant_stat_ac = participants_stats[participant]["Saccade Peak Acceleration"]
    # participant_stat_dec = participants_stats[participant]["Saccade Peak Deceleration"]
    if saccade_amp!="NA":
        sacacade_amplitude_count +=1
    if  saccade_peak_vel!="NA":
        saccade_peak_velocity_count +=1
    if saccade_amp !="NA" and not check_range(participant_stat_amp, saccade_amp):
        df.loc[index, "Saccade Amplitude"] = "NA"
        filtered_row['Saccade Amplitude'].append((index, participant))
    if saccade_peak_vel !="NA" and not check_range(participant_stat_vel, saccade_peak_vel):
        df.loc[index, "Saccade peak Velocity"] = "NA"
        filtered_row["Saccade Peak Velocity"].append((index, participant))
    # if saccade_amp !="NA" and not check_range(participant_stat_ac, saccade_peak_ac):
    #     row["Saccade Peak Acceleration"] = "NA"
    #     filtered_row["Saccade Peak Acceleration"].append(index)
    # if saccade_amp !="NA" and not check_range(participant_stat_dec, sccade_peak_dec):
    #     row["Saccade Peak Deceleration"] = "NA"
    #     filtered_row["Saccade Peak Deceleration"].append(index)

# Append processed rows to the combined_rows list
combined_rows.append(df)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(combined_rows, ignore_index=True)

# Sort by "Study Name" first in the specified order, then by "Respondent" numerically
study_order = ["Bridge 1", "Bridge 2", "Bridge 3", "Bridge 4"]
combined_df['Study Name'] = pd.Categorical(combined_df['Study Name'], categories=study_order, ordered=True)
combined_df = combined_df.sort_values(by=['Study Name', 'Respondent Name'])
        

Respondent 20022 still present after filtering AOI Label at data/saccade/SaccadeTable_easy1_second.csv
{'Saccade Amplitude': ['NA', 'NA', '0.92544799', '3.4434758', 'NA', '0.57982414', '1.21623425', 'NA', 'NA', 'NA', '1.27328824', '2.01638746', 'NA', '7.21633971', 'NA', '0.73965184', 'NA', '0.49975736', '0.67998746', 'NA', 'NA', '6.10021627', '1.90906727', '4.18815397', 'NA', 'NA', 'NA', '1.29234572', 'NA', 'NA', 'NA', '3.75715878', '0.59761714', 'NA', 'NA', '17.78168628', '3.26966941', 'NA', '1.04280893', 'NA', 'NA', '2.70374568', '19.46891175', '20.21086205', 'NA', 'NA', '2.06470267', '0.23464685', 'NA', 'NA', 'NA', '1.81220047', '5.43609861', '8.21690475', 'NA', 'NA', '0.52556569', 'NA', 'NA', '1.74151635', '0.20300514', '0.03622319', '2.27144317', 'NA', 'NA', '1.53728133', '0.16182597', '0.75168673', '0.7918555', '0.44034693', '0.58129926', '1.22715848', 'NA', '0.95569841', '0.41086927', '0.36692916', 'NA', '0.63776447', '1.30550218', 'NA', 'NA', 'NA', '0.60230954', '2.4378683', '0

In [96]:
# List unique values under the "AOI Label" column
unique_aoi_labels = combined_df["AOI Label"].unique()

# Print the unique values
print(unique_aoi_labels)

print(len(combined_df))

['Crack 1 Hit' 'Crack 2 Hit' 'Crack 3 Hit' 'Crack 4 Hit' 'Crack 5 Hit'
 'Base 1' 'Crack 12 Hit' 'Base 2' 'Crack 11 Hit' 'Crack 9 Hit'
 'Crack 8 Miss' 'Crack 7 Miss' 'Crack 6 HIt' 'Base 4' 'Crack 15 Hit'
 'Crack 13 Hit' 'Crack 16 HIt' 'Base 5' 'Crack 18 Hit' 'Crack 20 Hit'
 'Crack 19 Hit' 'Crack 14 Hit' 'Base 3' 'Crack 17 Hit' 'Crack 7 Hit'
 'Crack 6 Miss' 'Crack 8 Hit' 'Crack 10 Miss' 'Crack 16 Miss'
 'Crack 20 Miss' 'Crack 16 Hit' 'Crack 6 Hit' 'Crack 10 Hit'
 'Crack 14 Miss' 'Crack 17 Miss' 'Crack 18 Miss' 'FA 1' 'Crack 5 Miss'
 'Crack 12 Miss' 'Crack 15 Miss' 'Crack 8 hit' 'Crack 9 Miss'
 'Crack 19 Miss' 'Crack 4 Miss' 'Crack 1 Miss' 'Crack 2 Miss'
 'Crack 11 Miss' 'Crack 3 Miss' 'Crack 2 HIt' 'Crack 13 Miss']
54954


In [97]:
print(filtered_row)
print("percentage of saccade ampitude filtered: ", len(filtered_row['Saccade Amplitude'])/sacacade_amplitude_count)
print("percentage of peak velocity filtered: ", len(filtered_row['Saccade Peak Velocity'])/saccade_peak_velocity_count)

{'Saccade Amplitude': [(1087, 20017), (1091, 20017), (21184, 20021), (21185, 20021), (28502, 20023), (34048, 20024), (37032, 20024), (37045, 20024), (37336, 20024), (37601, 20024), (63111, 20029), (63912, 20029), (67471, 20030), (67476, 20030), (67478, 20030), (69270, 20030), (69611, 20030), (72000, 20031), (73589, 20031), (77185, 20032), (77504, 20032), (77511, 20032), (77669, 20032), (77670, 20032), (78391, 20032), (86054, 20034), (86914, 20034), (87026, 20034), (88100, 20034), (92007, 20034), (94731, 20034), (94761, 20034), (94764, 20034), (95547, 20034), (99172, 20035), (99848, 20035), (101120, 20035), (103152, 20036), (111563, 20037), (111572, 20037), (111573, 20037), (117553, 20038), (120297, 20038), (120334, 20038), (138026, 20042), (138995, 20042), (140592, 20043), (140949, 20043), (141391, 20043), (144321, 20044), (144832, 20044), (146387, 20044), (154192, 20046), (154807, 20046), (155538, 20046), (157156, 20046)], 'Saccade Peak Acceleration': [], 'Saccade Peak Deceleration': 

In [98]:
#Read timecard.csv
combined_timecard_df= pd.read_csv('combined_timecard.csv', skiprows=0)
print(combined_timecard_df.columns)

combined_df= pd.read_csv('combined_filtered_saccade_table.csv', skiprows=0)
combined_df['weight'] = 1
print(combined_df.columns)

combined_df = pd.read_csv('combined_filtered_saccade_table.csv')
unique_respondents_bridge_1 = combined_df[combined_df["Study Name"] == "Bridge 1"]["Respondent Name"].unique()

print('Bridge 1: ', unique_respondents_bridge_1)

unique_respondents_bridge_2 = combined_df[combined_df["Study Name"] == "Bridge 2"]["Respondent Name"].unique()

print('Bridge 2: ', unique_respondents_bridge_2)

unique_respondents_bridge_3 = combined_df[combined_df["Study Name"] == "Bridge 3"]["Respondent Name"].unique()

print('Bridge 3: ', unique_respondents_bridge_3)
unique_respondents_bridge_4 = combined_df[combined_df["Study Name"] == "Bridge 4"]["Respondent Name"].unique()

print('Bridge 4: ', unique_respondents_bridge_4)

Index(['Study Name', 'Respondent', 'Start', 'End', 'Label'], dtype='object')
Index(['Study Name', 'Respondent Name', 'Gaze Calibration', 'Stimulus Start',
       'Stimulus Duration', 'Saccade Index', 'Saccade Index by Stimulus',
       'Saccade Start', 'Saccade End', 'Saccade Duration', 'Saccade Amplitude',
       'Saccade Peak Velocity', 'Saccade Peak Acceleration',
       'Saccade Peak Deceleration', 'Saccade Direction', 'Saccade Type',
       'AOI Intersections', 'AOI Label', 'AOI Type', 'AOI Group',
       'AOI Instance', 'AOI Instance Start', 'AOI Instance Duration',
       'Saccade Index by AOI', 'Dwell Index (on AOI)', 'Saccade peak Velocity',
       'weight'],
      dtype='object')
Bridge 1:  [20002 20004 20005 20008 20011 20012 20013 20014 20015 20016 20017 20019
 20021 20024 20025 20026 20027 20029 20030 20031 20032 20034 20035 20036
 20037 20038 20039 20041 20042 20043 20044 20045 20046]
Bridge 2:  [20002 20004 20005 20006 20008 20009 20010 20012 20013 20015 20016 20017
 200

In [94]:
# # Standardize columns by stripping whitespace and converting to lowercase for consistency
# combined_df["Study Name"] = combined_df["Study Name"].str.strip().str.lower()
# combined_df["Respondent Name"] = combined_df["Respondent Name"].astype(str).str.strip()
# combined_df["AOI Label"] = combined_df["AOI Label"].str.strip().str.lower()

# combined_timecard_df["Study Name"] = combined_timecard_df["Study Name"].str.strip().str.lower()
# combined_timecard_df["Respondent"] = combined_timecard_df["Respondent"].astype(str).str.strip()
# combined_timecard_df["Label"] = combined_timecard_df["Label"].str.strip().str.lower()

rows_to_drop =[]
half_weight=0
# Iterate through each row in the DataFrame
for index, row in combined_df.iterrows():
    study = row["Study Name"]
    participant = row["Respondent Name"]
    aoiLabel = row["AOI Label"]
    saccade_start = row["Saccade Start"]
    saccade_end = row["Saccade End"]
    
    time_row = combined_timecard_df[(combined_timecard_df["Respondent"]==participant) & (combined_timecard_df["Label"]==aoiLabel) & (combined_timecard_df["Study Name"]==study)] #There should be a unique match
    
    if time_row.empty:
        print("Row empty: ", participant, aoiLabel, study) #This shouldn't happen
    if time_row.shape[0] > 1:
        print("Multiple rows: ", participant, aoiLabel, study) #This shouldn't happen
        print(time_row)
    
    start = time_row["Start"].values[0]
    end = time_row["End"].values[0]
    
    #drop the rows if saccade is outside the range completely
    if saccade_end < start or saccade_start > end:
        rows_to_drop.append(index)
    
    #saccade is partially covered by time window
    if saccade_start < start or saccade_end > end: 
        half_weight +=1
        combined_df.loc[index, 'weight'] = 0.5
        
    

In [ ]:
print(len(rows_to_drop))
print(half_weight)

24471
24926


In [ ]:
# Drop the rows from the original DataFrame
df_filtered = combined_df.drop(rows_to_drop)

# Count the number of rows where 'weight' equals 0.5
count = df_filtered[df_filtered['weight'] == 0.5].shape[0]

# Print the result
print(f'Number of rows where weight is 0.5: {count}')

# Save the modified DataFrame to a new CSV file (optional)
df_filtered.to_csv('combined_filtered_SaccadeTable.csv', index=False)


Number of rows where weight is 0.5: 455


In [ ]:
processed_combined_df = pd.read_csv('combined_filtered_SaccadeTable.csv')
print(processed_combined_df.head())

  Study Name  Respondent Name Gaze Calibration  Stimulus Start  \
0   Bridge 1            20002        Excellent       1534.4453   
1   Bridge 1            20002        Excellent       1534.4453   
2   Bridge 1            20002        Excellent       1534.4453   
3   Bridge 1            20002        Excellent       1534.4453   
4   Bridge 1            20002        Excellent       1534.4453   

   Stimulus Duration  Saccade Index  Saccade Index by Stimulus  Saccade Start  \
0          683570.31            474                        468     106287.310   
1          683570.31            475                        469     106487.371   
2          683570.31            479                        473     106845.642   
3          683570.31            480                        474     106970.664   
4          683570.31            481                        475     106995.645   

   Saccade End  Saccade Duration  ...    AOI Label     AOI Type  AOI Group  \
0   106320.635           33.3256  ... 

In [ ]:
#Inspect
print(processed_combined_df.columns)
# List unique values under the "Respondent Name" column
unique_respondents = processed_combined_df["Respondent Name"].unique()

# Print the unique values
print(unique_respondents)

# List unique values under the "AOI Group" column
unique_aoi_groups = processed_combined_df["AOI Group"].unique()

# Print the unique values
print(unique_aoi_groups)

# List unique values under the "AOI Instance" column
unique_aoi_instance_start = processed_combined_df["AOI Instance"].unique()

# Print the unique values
print(unique_aoi_instance_start)

print(len(processed_combined_df))

Index(['Study Name', 'Respondent Name', 'Gaze Calibration', 'Stimulus Start',
       'Stimulus Duration', 'Saccade Index', 'Saccade Index by Stimulus',
       'Saccade Start', 'Saccade End', 'Saccade Duration', 'Saccade Amplitude',
       'Saccade Peak Velocity', 'Saccade Peak Acceleration',
       'Saccade Peak Deceleration', 'Saccade Direction', 'Saccade Type',
       'AOI Intersections', 'AOI Label', 'AOI Type', 'AOI Group',
       'AOI Instance', 'AOI Instance Start', 'AOI Instance Duration',
       'Saccade Index by AOI', 'Dwell Index (on AOI)', 'Saccade peak Velocity',
       'weight'],
      dtype='object')
[20002 20004 20005 20008 20011 20012 20013 20014 20015 20016 20017 20019
 20021 20024 20025 20026 20027 20029 20030 20031 20032 20034 20035 20036
 20037 20038 20039 20041 20042 20043 20044 20045 20046 20006 20009 20010
 20023 20003 20007 20020 20022]
['Hits' 'Misses' 'Base' 'Hit' 'Miss' 'Bases' nan]
[1 3 2]
30483


In [43]:
#Next,we process the filtered table.
#Drop Gaze Calibration, Stimulus Start, Stimulus Duration, Saccade Index, Saccade Index by Stimulus, Saccade Type, Saccade Index by AOI, Dwell Index (on AOI), AOI Instance Duration, AOI Instance Start, AOI Type, AOI Intersections
#Drop Saccade Direction
#Saccade duration, amplitude, peak velocity, peak acceleration, peak deceleration -> mean, median, standard deviation
#Create a new column counting a number of saccade

#Drop columns
columns_to_drop = ["Gaze Calibration", "Stimulus Start", "Stimulus Duration", "Saccade Start", "Saccade End", "Saccade Index", "Saccade Index by Stimulus", "Saccade Type", "Saccade Index by AOI", "Dwell Index (on AOI)",
                   "AOI Instance Duration", "AOI Instance Start", "AOI Type", "AOI Intersections", "Saccade Direction", "AOI Instance"]

# Drop columns only if they exist in the DataFrame
processed_df = processed_combined_df.drop([col for col in columns_to_drop if col in df.columns], axis='columns')

In [44]:
#Calculate the aggrevated stats
# columns_to_aggrevate = ["Saccade Duration", "Saccade Amplitude", "Saccade Peak Velocity", "Saccade Peak Acceleration", 'Saccade Peak Deceleration']
# aggrevate_dict = {col: ["mean", "median", "std"] for col in columns_to_aggrevate}

In [45]:
row_count = 0
amplitude_count = 0

for index, row in processed_combined_df.iterrows():
    row_count += 1
    if pd.isna(row["Saccade Amplitude"]):  # Check if the value is NaN
        amplitude_count += 1

print(f"NaN Percentage: {amplitude_count / row_count * 100:.2f}%")

NaN Percentage: 40.14%


In [46]:
#Defining custom functions to calculate weighted_mean and weighted_std
def weighted_mean(values, weights):
    # Remove NaN values from both values and weights
    mask = ~pd.isna(values) & ~pd.isna(weights)
    valid_values = values[mask].astype(float)
    valid_weights = weights[mask].astype(float)
    
    if len(valid_values) > 0:
        return np.average(valid_values, weights=valid_weights)
    else:
        return np.nan  # Return NaN if no valid data remains

def weighted_std(values, weights):
    # Remove NaN values from both values and weights
    mask = ~pd.isna(values) & ~pd.isna(weights)
    valid_values = values[mask].astype(float)
    valid_weights = weights[mask].astype(float)
    
    if len(valid_values) > 0:
        average = weighted_mean(valid_values, valid_weights)
        variance = np.average((valid_values - average) ** 2, weights=valid_weights)
        return np.sqrt(variance)
    else:
        return np.nan  # Return NaN if no valid data remains

def weighted_count(values):
    return np.sum(values)

I need clarification on what NA means for those columns above

In [47]:
#Calculate statistics for each group (AOI label + participant)
def custom_agg(group):
    return pd.Series({
        'Saccade Duration Mean': weighted_mean(group['Saccade Duration'], group['weight']),
        'Saccade Duration Std': weighted_std(group['Saccade Duration'], group['weight']),
        'Saccade Duration Median': group['Saccade Duration'].median(),
        'Saccade Amplitude Mean': weighted_mean(group['Saccade Amplitude'], group['weight']),
        'Saccade Amplitude Std': weighted_std(group['Saccade Amplitude'], group['weight']),
        'Saccade Amplitude Median': group['Saccade Amplitude'].median(),
        'Saccade Peak Velocity Mean': weighted_mean(group['Saccade Peak Velocity'], group['weight']),
        'Saccade Peak Velocity Std': weighted_std(group['Saccade Peak Velocity'], group['weight']),
        'Saccade Peak Velocity Median': group['Saccade Peak Velocity'].median(),
        'Saccade Peak Acceleration Mean': weighted_mean(group['Saccade Peak Acceleration'], group['weight']),
        'Saccade Peak Acceleration Std': weighted_std(group['Saccade Peak Acceleration'], group['weight']),
        'Saccade Peak Acceleration Median': group['Saccade Peak Acceleration'].median(),
        'Saccade Peak Deceleration Mean': weighted_mean(group['Saccade Peak Deceleration'], group['weight']),
        'Saccade Peak Deceleration Std': weighted_std(group['Saccade Peak Deceleration'], group['weight']),
        'Saccade Peak Deceleration Median': group['Saccade Peak Deceleration'].median(),
        "Saccade Count" : weighted_count(group['weight']),
    })

In [49]:
# Save the modified DataFrame to a new CSV file (optional)
grouped_processed_combined_df = processed_combined_df.groupby(["Respondent Name", "AOI Label", "Study Name"]).apply(custom_agg).reset_index()

# # Flatten the multi-level column names
# grouped_processed_df.columns = [' '.join(col).strip() for col in grouped_processed_df.columns.values]

grouped_processed_combined_df['Study Name'] = pd.Categorical(grouped_processed_combined_df['Study Name'], categories=study_order, ordered=True)

grouped_processed_combined_df.to_csv('combined_processed_SaccadeTable.csv', index=False)

In [55]:
#Verify
processed_combined_df = pd.read_csv('combined_processed_SaccadeTable.csv')
unique_respondents_bridge_1 = processed_combined_df[processed_combined_df["Study Name"] == "Bridge 1"]["Respondent Name"].unique()

print('Bridge 1: ', unique_respondents_bridge_1)

unique_respondents_bridge_2 = processed_combined_df[processed_combined_df["Study Name"] == "Bridge 2"]["Respondent Name"].unique()

print('Bridge 2: ', unique_respondents_bridge_2)

unique_respondents_bridge_3 = processed_combined_df[processed_combined_df["Study Name"] == "Bridge 3"]["Respondent Name"].unique()

print('Bridge 3: ', unique_respondents_bridge_3)
unique_respondents_bridge_4 = processed_combined_df[processed_combined_df["Study Name"] == "Bridge 4"]["Respondent Name"].unique()

print('Bridge 4: ', unique_respondents_bridge_4)

# participant1 = list(unique_respondents_bridge_1)
# participant2 = list(unique_respondents_bridge_2)
# participant3 = list(unique_respondents_bridge_3)
# participant4 = list(unique_respondents_bridge_4)
# print(participant1)
# print(participant2)
# print(participant3)
# print(participant4)

Bridge 1:  [20002 20004 20005 20008 20011 20012 20013 20014 20015 20016 20017 20019
 20021 20024 20025 20026 20027 20029 20030 20031 20032 20034 20035 20036
 20037 20038 20039 20041 20042 20043 20044 20045 20046]
Bridge 2:  [20002 20004 20005 20006 20008 20009 20010 20012 20013 20015 20016 20017
 20021 20023 20024 20026 20029 20030 20031 20032 20034 20035 20036 20037
 20038 20039 20042 20043 20044 20046]
Bridge 3:  [20002 20003 20004 20005 20006 20007 20008 20009 20012 20015 20016 20017
 20019 20020 20022 20023 20024 20026 20027 20030 20032 20034 20035 20036
 20037 20038 20039 20041 20042 20043 20044 20045 20046]
Bridge 4:  [20002 20004 20005 20006 20009 20010 20011 20015 20016 20017 20019 20021
 20023 20024 20025 20026 20027 20029 20030 20031 20032 20034 20035 20036
 20037 20038 20039 20042 20043 20044 20045 20046]
[20002, 20004, 20005, 20008, 20011, 20012, 20013, 20014, 20015, 20016, 20017, 20019, 20021, 20024, 20025, 20026, 20027, 20029, 20030, 20031, 20032, 20034, 20035, 20036, 200